# Imports

In [89]:
import numpy as np
import pandas as pd
from characters.sparkle import Sparkle
from characters.pela import Pela
from characters.fu_xuan import Fu_Xuan
from characters.dhil import DHIL
from characters.luocha import Luocha
from characters.tingyun import Tingyun
from utils.arena import Arena
from utils.data_table import Action_Table, Qingque_Probability_Table

# User Imputs

In [90]:
dhil_speed = 98

In [91]:
pela_speed = 134

In [92]:
tingyun_speed = 161

In [93]:
luocha_speed = 134

In [94]:
sparkle_speed = 160.1

In [95]:
fu_xuan_speed = 134
fu_xuan_technique = True

# Simulations

## DHIL E0S0 Luocha Pela Sparkle

### Simulation

In [96]:
def initialize_dhil():
  arena = Arena()
  DH = DHIL(arena = arena, Spd = dhil_speed)
  PL = Pela(arena = arena, Spd = pela_speed)
  LCH = Luocha(arena = arena, Spd = luocha_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, PL, LCH

In [97]:
action_dataframe = Action_Table()

In [98]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, pl, lch = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, pl, lch]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if lch.action_gauge == 0:
        
        #Luocha's actions
            if verbose:
                print("Luocha takes a turn")
            action_df.add_row(arena, lch, "Basic", 1)
            lch.basic(verbose=verbose)
            lch.update_buffs()
            lch.action_reset()

        elif pl.action_gauge == 0:
        
        #Pela's actions
            if verbose:
                print("Silver Wolf takes a turn")
            action_df.add_row(arena, pl, "Basic", 1)
            pl.basic(verbose=verbose)
            pl.update_buffs()
            pl.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose, s1= False)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [99]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


### Results

In [100]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Sparkle,0.0,62.460962,6,55.00,Skill,-1
1,Dan Heng IL,0.0,62.560962,5,70.00,Enhanced Basic Lvl 3,-3
2,Luocha,0.0,74.626866,2,55.00,Basic,1
3,Pela,0.0,74.626866,3,55.00,Basic,1
4,Sparkle,0.0,124.921924,4,90.82,Skill,-1
5,Sparkle,0.0,124.921924,3,126.64,Ultimate,4
6,Dan Heng IL,0.0,125.021924,7,110.00,Enhanced Basic Lvl 3,-3
7,Dan Heng IL,0.0,125.021924,4,150.00,Ultimate,0
8,Luocha,0.0,134.328358,4,78.88,Basic,1
9,Pela,0.0,149.253731,5,75.00,Basic,1


In [101]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
1,Dan Heng IL,0.0,62.560962,5,70.0,Enhanced Basic Lvl 3,-3
6,Dan Heng IL,0.0,125.021924,7,110.0,Enhanced Basic Lvl 3,-3
7,Dan Heng IL,0.0,125.021924,4,150.0,Ultimate,0
11,Dan Heng IL,1.0,187.482886,5,5.0,Enhanced Basic Lvl 3,-1
15,Dan Heng IL,1.0,249.943848,5,45.0,Enhanced Basic Lvl 3,-3
20,Dan Heng IL,2.0,312.404809,7,85.0,Enhanced Basic Lvl 3,-3
25,Dan Heng IL,3.0,374.865771,6,125.0,Enhanced Basic Lvl 3,-3
26,Dan Heng IL,3.0,374.865771,3,165.0,Ultimate,0
29,Dan Heng IL,3.0,437.326733,3,5.0,Enhanced Basic Lvl 3,-1
34,Dan Heng IL,4.0,499.787695,7,45.0,Enhanced Basic Lvl 3,-3


## DHIL E0S1 Luocha Pela Sparkle

### Simulation

In [102]:
def initialize_dhil():
  arena = Arena()
  DH = DHIL(arena = arena, Spd = dhil_speed, s1 = True)
  PL = Pela(arena = arena, Spd = pela_speed)
  LCH = Luocha(arena = arena, Spd = luocha_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, PL, LCH

In [103]:
action_dataframe = Action_Table()

In [104]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, pl, lch = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, pl, lch]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if lch.action_gauge == 0:
        
        #Luocha's actions
            if verbose:
                print("Luocha takes a turn")
            action_df.add_row(arena, lch, "Basic", 1)
            lch.basic(verbose=verbose)
            lch.update_buffs()
            lch.action_reset()


        elif pl.action_gauge == 0:
        
        #Pela's actions
            if verbose:
                print("Silver Wolf takes a turn")
            action_df.add_row(arena, pl, "Basic", 1)
            pl.basic(verbose=verbose)
            pl.update_buffs()
            pl.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose, s1 = True)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [105]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


### Results

In [106]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Sparkle,0.0,62.460962,6,55.00,Skill,-1
1,Dan Heng IL,0.0,62.560962,5,70.00,Enhanced Basic Lvl 3,-3
2,Luocha,0.0,74.626866,2,55.00,Basic,1
3,Pela,0.0,74.626866,3,55.00,Basic,1
4,Sparkle,0.0,124.921924,4,90.82,Skill,-1
5,Sparkle,0.0,124.921924,3,126.64,Ultimate,4
6,Dan Heng IL,0.0,125.021924,7,114.80,Enhanced Basic Lvl 3,-3
7,Dan Heng IL,0.0,125.021924,4,159.60,Ultimate,0
8,Luocha,0.0,134.328358,4,78.88,Basic,1
9,Pela,0.0,149.253731,5,75.00,Basic,1


In [107]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
1,Dan Heng IL,0.0,62.560962,5,70.0,Enhanced Basic Lvl 3,-3
6,Dan Heng IL,0.0,125.021924,7,114.8,Enhanced Basic Lvl 3,-3
7,Dan Heng IL,0.0,125.021924,4,159.6,Ultimate,0
11,Dan Heng IL,1.0,187.482886,5,5.6,Enhanced Basic Lvl 3,-1
15,Dan Heng IL,1.0,249.943848,5,50.4,Enhanced Basic Lvl 3,-3
20,Dan Heng IL,2.0,312.404809,7,95.2,Enhanced Basic Lvl 3,-3
21,Dan Heng IL,2.0,312.404809,4,140.0,Ultimate,0
26,Dan Heng IL,3.0,374.865771,6,5.6,Enhanced Basic Lvl 3,-1
29,Dan Heng IL,3.0,437.326733,5,50.4,Enhanced Basic Lvl 3,-3
34,Dan Heng IL,4.0,499.787695,7,95.2,Enhanced Basic Lvl 3,-3


## DHIL E0S0 Luocha Tingyun Sparkle

### Simulation

In [108]:
def initialize_dhil():
  arena = Arena()
  DH = DHIL(arena = arena, Spd = dhil_speed)
  TY = Tingyun(arena = arena, recipient= DH, Spd = tingyun_speed)
  LCH = Luocha(arena = arena, Spd = luocha_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, TY, LCH

In [109]:
action_dataframe = Action_Table()

In [110]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, ty, lch = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, ty, lch]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if lch.action_gauge == 0:
        
        #Luocha's actions
            if verbose:
                print("Luocha takes a turn")
            action_df.add_row(arena, lch, "Basic", 1)

            lch.update_buffs()
            lch.action_reset()

        elif ty.action_gauge == 0:
        
        #Tingyun's actions
            if verbose:
                print("Tingyun takes a turn")
            ty.energy +=5*ty.energy_regen
            if arena.sp > 0 and ty.skill_counter <= 0:
                action_df.add_row(arena, ty, "Skill", -1)
                ty.skill(verbose=verbose)
            else:          
                action_df.add_row(arena, ty, "Basic", 1)
                ty.basic(verbose=verbose)
            if ty.max_energy <= ty.energy:
                action_df.add_row(arena, ty, "Ultimate", 0)
                ty.ult(verbose=verbose)
            ty.update_buffs()
            ty.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose, s1= False)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [111]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


### Results

In [112]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
ty_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Tingyun"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Tingyun,0.0,62.111801,6,71.220,Skill,-1
1,Sparkle,0.0,62.460962,5,55.000,Skill,-1
2,Dan Heng IL,0.0,62.560962,4,70.000,Enhanced Basic Lvl 3,-3
3,Luocha,0.0,74.626866,1,55.000,Basic,1
4,Tingyun,0.0,124.223602,1,124.712,Basic,1
5,Tingyun,0.0,124.223602,2,159.544,Ultimate,0
6,Sparkle,0.0,124.921924,2,90.820,Skill,-1
7,Sparkle,0.0,124.921924,1,126.640,Ultimate,4
8,Dan Heng IL,0.0,125.021924,5,160.000,Ultimate,0
9,Dan Heng IL,0.0,125.021924,5,5.000,Enhanced Basic Lvl 3,-1


In [113]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
2,Dan Heng IL,0.0,62.560962,4,70.0,Enhanced Basic Lvl 3,-3
8,Dan Heng IL,0.0,125.021924,5,160.0,Ultimate,0
9,Dan Heng IL,0.0,125.021924,5,5.0,Enhanced Basic Lvl 3,-1
13,Dan Heng IL,1.0,187.482886,4,45.0,Enhanced Basic Lvl 3,-3
17,Dan Heng IL,2.0,289.523702,1,85.0,Enhanced Basic Lvl 1,-1
26,Dan Heng IL,3.0,374.865771,6,165.0,Ultimate,0
27,Dan Heng IL,3.0,374.865771,6,5.0,Enhanced Basic Lvl 3,-1
30,Dan Heng IL,3.0,437.326733,3,45.0,Enhanced Basic Lvl 3,-3
37,Dan Heng IL,4.0,539.367550,6,135.0,Enhanced Basic Lvl 3,-3
38,Dan Heng IL,4.0,539.367550,3,175.0,Ultimate,0


In [114]:
ty_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Tingyun,0.0,62.111801,6,71.220,Skill,-1
4,Tingyun,0.0,124.223602,1,124.712,Basic,1
5,Tingyun,0.0,124.223602,2,159.544,Ultimate,0
11,Tingyun,1.0,186.335404,4,12.440,Basic,1
15,Tingyun,1.0,248.447205,1,53.492,Skill,-1
19,Tingyun,2.0,310.559006,0,106.984,Basic,1
20,Tingyun,2.0,310.559006,1,141.816,Ultimate,0
23,Tingyun,3.0,372.670807,6,12.440,Basic,1
28,Tingyun,3.0,434.782609,5,53.492,Skill,-1
32,Tingyun,4.0,496.894410,0,106.984,Basic,1


## DHIL E0S1 Luocha Pela Sparkle

In [115]:
def initialize_dhil():
  arena = Arena()
  DH = DHIL(arena = arena, Spd = dhil_speed, s1 = True)
  PL = Pela(arena = arena, Spd = pela_speed)
  LCH = Luocha(arena = arena, Spd = luocha_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, PL, LCH

In [116]:
action_dataframe = Action_Table()

In [117]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, pl, lch = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, pl, lch]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if lch.action_gauge == 0:
        
        #Luocha's actions
            if verbose:
                print("Luocha takes a turn")
            action_df.add_row(arena, lch, "Basic", 1)

            lch.update_buffs()
            lch.action_reset()

        elif pl.action_gauge == 0:
        
        #Pela's actions
            if verbose:
                print("Silver Wolf takes a turn")
            action_df.add_row(arena, pl, "Basic", 1)
            pl.basic(verbose=verbose)
            pl.update_buffs()
            pl.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose, s1 = True)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [118]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


In [119]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Sparkle,0.0,62.460962,6,55.00,Skill,-1
1,Dan Heng IL,0.0,62.560962,5,70.00,Enhanced Basic Lvl 3,-3
2,Luocha,0.0,74.626866,2,55.00,Basic,1
3,Pela,0.0,74.626866,2,55.00,Basic,1
4,Sparkle,0.0,124.921924,3,90.82,Skill,-1
5,Sparkle,0.0,124.921924,2,126.64,Ultimate,4
6,Dan Heng IL,0.0,125.021924,6,114.80,Enhanced Basic Lvl 3,-3
7,Dan Heng IL,0.0,125.021924,3,159.60,Ultimate,0
8,Luocha,0.0,149.253731,3,55.00,Basic,1
9,Pela,0.0,149.253731,3,75.00,Basic,1


In [120]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
1,Dan Heng IL,0.0,62.560962,5,70.0,Enhanced Basic Lvl 3,-3
6,Dan Heng IL,0.0,125.021924,6,114.8,Enhanced Basic Lvl 3,-3
7,Dan Heng IL,0.0,125.021924,3,159.6,Ultimate,0
11,Dan Heng IL,1.0,187.482886,3,5.6,Enhanced Basic Lvl 3,-1
15,Dan Heng IL,1.0,249.943848,2,50.4,Enhanced Basic Lvl 2,-2
20,Dan Heng IL,3.0,351.984664,6,89.6,Enhanced Basic Lvl 3,-3
24,Dan Heng IL,3.0,403.105072,3,134.4,Enhanced Basic Lvl 3,-3
25,Dan Heng IL,3.0,403.105072,0,179.2,Ultimate,0
31,Dan Heng IL,4.0,499.787695,5,5.6,Enhanced Basic Lvl 3,-1
35,Dan Heng IL,5.0,562.248657,4,50.4,Enhanced Basic Lvl 3,-3


## DHIL E0S0 Luocha Pela Sparkle

In [121]:
def initialize_dhil():
  arena = Arena()
  DH = DHIL(arena = arena, Spd = dhil_speed)
  PL = Pela(arena = arena, Spd = pela_speed)
  LCH = Luocha(arena = arena, Spd = luocha_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, PL, LCH

In [122]:
action_dataframe = Action_Table()

In [123]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, pl, lch = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, pl, lch]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if lch.action_gauge == 0:
        
        #Luocha's actions
            if verbose:
                print("Luocha takes a turn")
            action_df.add_row(arena, lch, "Basic", 1)

            lch.update_buffs()
            lch.action_reset()

        elif pl.action_gauge == 0:
        
        #Pela's actions
            if verbose:
                print("Silver Wolf takes a turn")
            action_df.add_row(arena, pl, "Basic", 1)
            pl.basic(verbose=verbose)
            pl.update_buffs()
            pl.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose, s1= False)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [124]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


In [125]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Sparkle,0.0,62.460962,6,55.00,Skill,-1
1,Dan Heng IL,0.0,62.560962,5,70.00,Enhanced Basic Lvl 3,-3
2,Luocha,0.0,74.626866,2,55.00,Basic,1
3,Pela,0.0,74.626866,2,55.00,Basic,1
4,Sparkle,0.0,124.921924,3,90.82,Skill,-1
5,Sparkle,0.0,124.921924,2,126.64,Ultimate,4
6,Dan Heng IL,0.0,125.021924,6,110.00,Enhanced Basic Lvl 3,-3
7,Dan Heng IL,0.0,125.021924,3,150.00,Ultimate,0
8,Luocha,0.0,149.253731,3,55.00,Basic,1
9,Pela,0.0,149.253731,3,75.00,Basic,1


In [126]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
1,Dan Heng IL,0.0,62.560962,5,70.0,Enhanced Basic Lvl 3,-3
6,Dan Heng IL,0.0,125.021924,6,110.0,Enhanced Basic Lvl 3,-3
7,Dan Heng IL,0.0,125.021924,3,150.0,Ultimate,0
11,Dan Heng IL,1.0,187.482886,3,5.0,Enhanced Basic Lvl 3,-1
15,Dan Heng IL,1.0,249.943848,2,45.0,Enhanced Basic Lvl 2,-2
20,Dan Heng IL,3.0,351.984664,6,80.0,Enhanced Basic Lvl 3,-3
24,Dan Heng IL,3.0,403.105072,3,120.0,Enhanced Basic Lvl 3,-3
25,Dan Heng IL,3.0,403.105072,0,160.0,Ultimate,0
31,Dan Heng IL,4.0,499.787695,5,5.0,Enhanced Basic Lvl 3,-1
35,Dan Heng IL,5.0,562.248657,4,45.0,Enhanced Basic Lvl 3,-3


## DHIL E0S1 Luocha Pela Sparkle

In [127]:
def initialize_dhil():
  arena = Arena()
  DH = DHIL(arena = arena, Spd = dhil_speed, s1 = True)
  PL = Pela(arena = arena, Spd = pela_speed)
  LCH = Luocha(arena = arena, Spd = luocha_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, PL, LCH

In [128]:
action_dataframe = Action_Table()

In [129]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, pl, lch = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, pl, lch]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if lch.action_gauge == 0:
        
        #Luocha's actions
            if verbose:
                print("Luocha takes a turn")
            action_df.add_row(arena, lch, "Basic", 1)

            lch.update_buffs()
            lch.action_reset()

        elif pl.action_gauge == 0:
        
        #Pela's actions
            if verbose:
                print("Silver Wolf takes a turn")
            action_df.add_row(arena, pl, "Basic", 1)
            pl.basic(verbose=verbose)
            pl.update_buffs()
            pl.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose, s1 = True)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [130]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


In [131]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Sparkle,0.0,62.460962,6,55.00,Skill,-1
1,Dan Heng IL,0.0,62.560962,5,70.00,Enhanced Basic Lvl 3,-3
2,Luocha,0.0,74.626866,2,55.00,Basic,1
3,Pela,0.0,74.626866,2,55.00,Basic,1
4,Sparkle,0.0,124.921924,3,90.82,Skill,-1
5,Sparkle,0.0,124.921924,2,126.64,Ultimate,4
6,Dan Heng IL,0.0,125.021924,6,114.80,Enhanced Basic Lvl 3,-3
7,Dan Heng IL,0.0,125.021924,3,159.60,Ultimate,0
8,Luocha,0.0,149.253731,3,55.00,Basic,1
9,Pela,0.0,149.253731,3,75.00,Basic,1


In [132]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
1,Dan Heng IL,0.0,62.560962,5,70.0,Enhanced Basic Lvl 3,-3
6,Dan Heng IL,0.0,125.021924,6,114.8,Enhanced Basic Lvl 3,-3
7,Dan Heng IL,0.0,125.021924,3,159.6,Ultimate,0
11,Dan Heng IL,1.0,187.482886,3,5.6,Enhanced Basic Lvl 3,-1
15,Dan Heng IL,1.0,249.943848,2,50.4,Enhanced Basic Lvl 2,-2
20,Dan Heng IL,3.0,351.984664,6,89.6,Enhanced Basic Lvl 3,-3
24,Dan Heng IL,3.0,403.105072,3,134.4,Enhanced Basic Lvl 3,-3
25,Dan Heng IL,3.0,403.105072,0,179.2,Ultimate,0
31,Dan Heng IL,4.0,499.787695,5,5.6,Enhanced Basic Lvl 3,-1
35,Dan Heng IL,5.0,562.248657,4,50.4,Enhanced Basic Lvl 3,-3
